In [2]:
%load_ext autoreload
%autoreload 2 
import numpy as np
from maze import get_actions
from maze import get_possible_actions
from maze import next_step
from maze import get_state
from maze import get_transition_probabilities
from maze import get_rewards
from maze import describe_action

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Problem

We have an agent in position `(0,0)`. The goal is get the agent to `(2,2)` using 4 possible movements: left, right, up and down. Agent will only get a reward if it gets to its goal.

In [27]:
matrix = np.zeros((3,3))

In [28]:
goal = (2, 2)
agent = (0, 0)
# 0 -> Left
# 1 -> Right
# 2 -> Up
# 3 -> Down
actions = np.array([0, 1, 2, 3])

### Markov decision process

If we model this problem as a MDP, each position the agent can take in the matrix is one state. From each state, the agent can only take certain actions (i.e if the agent is at `(0,1)`, equivelant to state `1` in the maze action `U` is impossible).

<img src="resources/imgs/MDP.png">

##### Transition probabilities

This tell us what are the probabilities to get from one state to another using a specific action. Each row is a state. Each column is an action. Each element in the matrix is an array, where each position represents one state, its value if the probability of moving from the state in the row to this state, with the action in the column. 

In [29]:
transition_probabilities = get_transition_probabilities(matrix, actions)

If the agent is on state `4` (which is the center state) and it takes action `Left`, what are the probabilities for it to get to any of the other states? 

In [30]:
transition_probabilities[4,0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0.])

It can only get to state `3` if it takes action `Left` with probability of `1`.

##### Possible actions

Possible actions an agent can take from one position. For instance on state `0` it can only either go `right` or `down`.

In [31]:
possible_actions = get_possible_actions(matrix)
possible_actions

[[1, 3],
 [0, 1, 3],
 [0, 3],
 [1, 2, 3],
 [0, 1, 2, 3],
 [0, 2, 3],
 [1, 2],
 [0, 1, 2],
 [0, 2]]

##### Rewards

Describe all possible actions an agent can take from a different place at the matrix.

In [32]:
rewards = get_rewards(matrix, actions, goal)

What are the rewards an agent can get if it is on state `4` and it moves left?

In [33]:
rewards[4,0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

Nothing! From this position, moving left the agent will get to state `3` and that is not our goal. However what happens if I am on state `5` and I take action `down` ?

In [34]:
rewards[5, 3]

array([0., 0., 0., 0., 0., 0., 0., 0., 1.])

I will get a reward of `1` if I end up on state `8`, which our goal.

### Q-Value Iteration algorithm

Now, let's run the `Q-Value Iteration algorithm` to see what is the best action we can take from each state.

In [35]:
gamma = 0.9 # Discount factor 

t_states = matrix.size
t_actions = actions.size
Q_values = np.zeros((t_states, t_actions))

for iteration in range(50):
    Q_prev = Q_values.copy()
    
    for s in range(t_states):
        for a in possible_actions[s]:
            Q_values[s,a] = np.sum([
                transition_probabilities[s,a,sp]
                * (rewards[s, a, sp] + gamma * np.max(Q_prev[sp]))
            for sp in range(t_states)])
Q_values

array([[0.        , 3.81242949, 0.        , 3.81242949],
       [3.42603276, 4.23603276, 0.        , 4.23603276],
       [3.81242949, 0.        , 0.        , 4.71242949],
       [0.        , 4.23603276, 3.42603276, 4.23603276],
       [3.81242949, 4.71242949, 3.81242949, 4.71242949],
       [4.23603276, 0.        , 4.23603276, 5.23603276],
       [0.        , 4.71242949, 3.81242949, 0.        ],
       [4.23603276, 5.23603276, 4.23603276, 0.        ],
       [4.71242949, 0.        , 4.71242949, 0.        ]])

Each row represents a state and each column represents an action. The highest value in an action per state indicates which is the best action to take if the agent is on that state. So, what are the best actions to take on every state?

In [36]:
best_actions = np.argmax(Q_values, axis=1)
best_actions

array([1, 1, 3, 1, 1, 3, 1, 1, 0])

In [37]:
[describe_action(a) for a in best_actions]

['Right', 'Right', 'Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Left']

This is the optimal policy for our MDP. 